# HTML解析入门及准备URL生成连续技
![for humans](https://requests-html.kennethreitz.org/_static/requests-html-logo.png#thumbnail)

*  本周主要内容：HTML解析（parse HTML）及准备URL生成连续技
*  上周主要内容：HTML解析（parse HTML）及Xpath实践
*  20春_Web数据挖掘_week03
*  电子讲义设计者：廖汉腾, 许智超
<br/>
<br/>

-----
## 复习

复习：上周内容，实践

* HTML解析（parse HTML）: requests-html  丶
* Xpath实践
* m.liepin.com 取工作牛肉

-----
## 本周内容及学习目标

本周内容聚焦在

<mark> 如何从一页开始有系统的找更多页的内容 </mark>

为此，我们需要学习

1. 拆解带有参数的URL，并再从query取出参数
   a. URL拆解: 使用 urllib.parse 解析 出query
   b. query拆解:  取出参数 成python字典
2. 有基底URL，加上参数字典，请求新网页连续技

我们除了继续学习解决上一周已开始面对的以下挑战：
![Xpath Axis](http://krum.rz.uni-mannheim.de/inet-2005/images/xpath-axis.gif)

### 旧目标
1. 使用 requests-html 爬取并存取网页文字档，查找[requests-html 中文文档](https://cncert.github.io/requests-html-doc-cn/#/)
2. 熟悉 [xpath 语法](https://www.w3cschool.cn/xpath/xpath-syntax.html)丶[xpath 节点](https://www.w3cschool.cn/xpath/xpath-nodes.html)
3. 使用 [xpath cheatsheet](https://devhints.io/xpath)
  * 在 Chrome Inspector 使用
  * 在 requests-html (Python) 使用
4. 简易使用 [pd.DataFrame](https://www.pypandas.cn/doc/getting_started/dsintro.html#dataframe)

### 新目标
这一周，学生将实践
* 猎聘PC版 liepin.com 取工作URL参数的牛肉
* 如何生成一连串新URL以进一步爬取数据




In [1]:
%%html
<style>
/* 本电子讲义使用之CSS */
div.code_cell {
    background-color: #e5f1fe;
}
div.cell.selected {
    background-color: #effee2;
    font-size: 2rem;
    line-height: 2.4rem;
}
div.cell.selected .rendered_html table {
    font-size: 2rem !important;
    line-height: 2.4rem !important;
}
.rendered_html pre code {
    background-color: #C4E4ff;   
    padding: 2px 25px;
}
.rendered_html pre {
    background-color: #99c9ff;
}
div.code_cell .CodeMirror {
    font-size: 2rem !important;
    line-height: 2.4rem !important;
}
.rendered_html img, .rendered_html svg {
    max-width: 60%;
    height: auto;
    float: right;
}

.rendered_html img[src*="#full"], .rendered_html svg[src*="#full"] {
    max-width: 100%;
    height: auto;
    float: none;
}

.rendered_html img[src*="#thumbnail"], .rendered_html svg[src*="#thumbnail"] {
    max-width: 15%;
    height: auto;
}

/* Gradient transparent - color - transparent */
hr {
    border: 0;
    border-bottom: 1px dashed #ccc;
}
.emoticon{
    font-size: 5rem;
    line-height: 4.4rem;
    text-align: center;
    vertical-align: middle;
}
.bg-split_apply_comine {
    width: 500px;     
    height: 300px;
    background: url('02_split-apply-comine_500x300.png') -10px -10px;
    float: right;
}
.bg-comine {
    width: 175px;
    height: 150px;
    background: url('02_split-apply-comine_500x300.png') -280px -80px;
    float: right;
}
.bg-apply {
    width: 155px;
    height: 225px;
    background: url('02_split-apply-comine_500x300.png') -160px -30px;
    float: right;
}
.bg-split {
    width: 205px;
    height: 225px;
    background: url('02_split-apply-comine_500x300.png') -10px -30px;
    float: right;
}
.break {
                   page-break-after: right; 
                   width:700px;
                   clear:both;
}
</style>

In [2]:
# 基本模块
import pandas as pd
from requests_html import HTMLSession

# 0. 上周加分作业解答

In [4]:
# C-1   单一页面
url = "https://m.liepin.com/zhaopin/?keyword=PRD"
session = HTMLSession()
r = session.get( url )

# C-5
# 难: '公司URL', '时间', '经验'

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath( \
    '//div[@class="job-card-wrap"]//div[@class="job-card"]')

# 作为xpath字典，键为我要抓的牛肉名称，值为xpath
dict_xpaths={ 
    'text': {
        '经验':      './/ul/li[time]/text()'
    },
    'text_content': {
        '职称':    './/ul/li/a[contains(@class,"job-name")]/span[@class="name-text"]', 
        '薪水':    './/ul/li/a[contains(@class,"job-name")]/following-sibling::span', 
        '公司地点':'.//ul/li/time/following-sibling::a',
        '公司名称': './/ul/li/a[contains(@class,"company-name")]', 
        '时间':    './/ul/li/time', 
    },
    'href': {
        '链结':    './/ul/li/a[contains(@class,"job-name")]', 
        '公司URL': './/ul/li/a[contains(@class,"company-name")]', 
    }
}

def get_e_text_content(_xpath_):
    # 高级列表推导
    暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
    return(暂存结果)

def get_e_text(_xpath_):
    # 高级列表推导
    暂存结果 = ["".join([x.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
    return(暂存结果)

def get_e_href(_xpath_):
    # 高级列表推导
    暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
               if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
               else "" for e in 主要元素]
    return(暂存结果)

# 只对主要元素下进行.xpath取值
数据字典 = dict()

数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

print ([len(v) for k,v in 数据字典.items()])  # 檢查

数据 = pd.DataFrame(数据字典)
数据.to_excel("20春_Web数据挖掘_week02_liepin.xlsx", sheet_name="搜查结果")
数据 

[60, 60, 60, 60, 60, 60, 60, 60]


,职称,薪水,公司地点,公司名称,时间,经验,链结,公司URL
0,旅游产品经理,12-20k·12薪,广州,前海爱讯科技(深圳)有限公司,19小时前,2年以上 统招本科,https://m.liepin.com/job/1926703515.shtml,https://m.liepin.com/company/8972310/
1,教育科技 软件产品经理,12-18k·12薪,广州,融捷投资控股集团,昨天,3年以上 统招本科,https://m.liepin.com/job/1922705123.shtml,https://m.liepin.com/company/8025674/
2,产品经理,12-18k·12薪,广州-海珠区,广州大白互联网科技有限公司,昨天,2年以上 本科及以上,https://m.liepin.com/job/1922402715.shtml,https://m.liepin.com/company/8695948/
3,实施经理,16-23k·12薪,广州-大沙,广东卓志供应链服务集团有限公司,2020-03-23,5年以上 统招本科,https://m.liepin.com/job/1924985573.shtml,https://m.liepin.com/company/9238204/
4,互联网产品经理,10-15k·12薪,广州-琶洲,广东车海洋环保科技有限公司,2020-03-20,3年以上 大专及以上,https://m.liepin.com/job/1917453193.shtml,https://m.liepin.com/company/9256869/
5,后台产品经理,10-20k·12薪,广州,广东南方新媒体股份有限公司,一个月前,3年以上 本科及以上,https://m.liepin.com/job/1925126353.shtml,https://m.liepin.com/company/7889168/
6,区块链产品经理,15-25k·12薪,广州-黄埔区,北京普瑞未来教育科技集团有限公司,一个月前,3年以上 大专及以上,https://m.liepin.com/job/1919835727.shtml,https://m.liepin.com/company/9989029/
7,高级产品经理,20-25k·13薪,广州,某软件开发企业,昨天,3年以上 大专及以上,https://m.liepin.com/a/18948933.shtml,
8,产品经理（电商系统）,25-40k·14薪,"广东,深圳,广州",知名跨境电商公司,昨天,3年以上 统招本科,https://m.liepin.com/a/18705133.shtml,
9,WMS产品经理,20-35k·14薪,广州,某知名跨境电商平台,一个月前,2年以上 学历不限,https://m.liepin.com/a/18963147.shtml,


In [5]:
print(数据字典)

{'职称': ['旅游产品经理 ', '教育科技 软件产品经理 ', '产品经理 ', '实施经理 ', '互联网产品经理 ', '后台产品经理 ', '区块链产品经理 ', '高级产品经理 ', '产品经理（电商系统） ', 'WMS产品经理 ', '产品经理（支付/后端） ', '产品总监 ', '产品专员 ', '产品助理 ', '产品经理 ', '产品经理 ', '供应链产品经理 ', '产品经理 ', '产品经理（校园招聘） ', 'WXG03-微信公众号小程序生活服务行业产品经理（广州） ', '高级业务架构师（数字化新零售） ', '产品经理（智能终端产品） ', '产品经理 ', '产品经理（营收） ', '产品经理-内容优化方向 ', '平台SDK产品经理 ', '账号产品经理 ', '产品经理（电商/社交方向） ', '资深产品经理（相机产品） ', '产品经理（用户体验改善） ', '产品经理 ', '高级产品经理 ', '产品专员 ', '高级产品经理(J10274) ', 'ATS需求分析师 ', '产品经理（临床科研） ', '产品经理-供应链金融 ', '直播产品经理 ', '产品经理（全自动运行方向） ', '产品经理（节能控制方向） ', '产品经理（智能运维方向） ', '产品经理（数字孪生方向） ', '产品经理 ', '数据产品经理 ', '直播营收产品运营经理 ', '产品策划经理 ', '产品经理（跨境业务） ', '内衣项目总经理 - 跨境电商出口行业 ', '产品经理（OA） ', '资深产品 ', 'APP产品经理（知识付费/在线教育） ', 'APP产品经理（知识付费/在线教育） ', '国际支付产品经理 ', 'crm高级产品经理 ', '产品总监 ', '数据中台产品经理 ', '商家履约产品经理 ', 'WMS产品经理 ', '资深产品经理（后台系统） ', 'wms产品经理 '], '薪水': ['12-20k·12薪', '12-18k·12薪', '12-18k·12薪', '16-23k·12薪', '10-15k·12薪', '10-20k·12薪', '15-25k·12薪', '20-25k·13薪', '25-40k·14薪', '20-35k·14薪', '10-20k·12薪', '50-

#### 小坑/小风格
* 代码某几行最后一个字符有 \，指的是什麽意思？
* 代码某几行最后一个字符有 \，为什麽要用？给机器还是人用的？
* 代码某几行最后一个字符有 \，若后面多了空白会怎麽样？

----
答案: 和机器说，此行代码未结束，下行继续，最主要是让**写**程序的人可以合法回车，目标主要是为了让**读**代码的人可以因为好的回车排版，更易懂代码的意义

#### 列表推?油 requests_html?


In [6]:
# Z-1 別人代碼
from requests_html import HTMLSession
session = HTMLSession()

url = 'https://search.jd.com/Search?keyword=x1&enc=utf-8&wq=x1&pvid=add52cb63f7e4887b5ba29406a5756ba'
r = session.get(url)
link_list = r.html.find('.gl-item')
for i in link_list:
    print(i.text)

￥9499.00
联想ThinkPad
X1
Carbon 2019（20CD）英特尔酷睿i5 14英寸轻薄笔记本电脑（i5-8265U 8G 512GSSD FHD）黑 【加35元即可换购无线鼠标】商品加入购物车，查看商品上方“换购”，PC端点击“换购商品”，APP端点击“去换购”即可参与！无线鼠标E3黑红双色可换壳
二手有售
ThinkPad京东自营旗舰店
自营 满赠
对比 关注 加入购物车
￥9999.00
联想ThinkPad
X1
Carbon 2019（01CD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-10210U 8G 512GSSD FHD)黑 【加35元即可换购无线鼠标】商品加入购物车，查看商品上方“换购”，PC端点击“换购商品”，APP端点击“去换购”即可参与！无线鼠标E3黑红双色可换壳
二手有售
ThinkPad京东自营旗舰店
自营 赠
对比 关注 加入购物车
￥16999.00
联想ThinkPad
X1
Carbon 2019（04CD）英特尔酷睿i7 14英寸轻薄笔记本电脑(i7-10710U 16G 1TSSD WQHD)4G版 【加35元即可换购无线鼠标】商品加入购物车，查看商品上方“换购”，PC端点击“换购商品”，APP端点击“去换购”即可参与！无线鼠标E3黑红双色可换壳
二手有售
ThinkPad京东自营旗舰店
自营 赠
对比 关注 加入购物车
￥14799.00
联想ThinkPad
X1
Carbon 2019（02CD）英特尔酷睿i7 14英寸轻薄笔记本电脑(i7-10710U 16G 512GSSD WQHD)4G版 【加35元即可换购无线鼠标】商品加入购物车，查看商品上方“换购”，PC端点击“换购商品”，APP端点击“去换购”即可参与！无线鼠标E3黑红双色可换壳
ThinkPad京东自营旗舰店
自营 赠
对比 关注 加入购物车
￥13999.00
联想ThinkPad
X1
Carbon 2019（05CD）英特尔酷睿i7 14英寸轻薄笔记本电脑(i7-10710U 16G 512SSD FHD)4G版 【加35元即可换购无线鼠标】商品加入购物车，查看商品上方“换购”，PC端点击“换购商品”，APP端点击“去换购”即可参与！无线鼠标E3黑红双色可换壳
二手有售
ThinkPad京东自营旗舰店
自营 赠
对比 关注 

In [7]:
# Z-1 別人代碼 列表推導
'''
link_list = r.html.find('.gl-item')
列表暫存 = list()
for i in link_list:
    列表暫存.append(i.text)
    #print(i.text)
'''
[i.text for i in link_list]

['￥9499.00\n联想ThinkPad\nX1\nCarbon 2019（20CD）英特尔酷睿i5 14英寸轻薄笔记本电脑（i5-8265U 8G 512GSSD FHD）黑 【加35元即可换购无线鼠标】商品加入购物车，查看商品上方“换购”，PC端点击“换购商品”，APP端点击“去换购”即可参与！无线鼠标E3黑红双色可换壳\n二手有售\nThinkPad京东自营旗舰店\n自营 满赠\n对比 关注 加入购物车',
 '￥9999.00\n联想ThinkPad\nX1\nCarbon 2019（01CD）英特尔酷睿i5 14英寸轻薄笔记本电脑(i5-10210U 8G 512GSSD FHD)黑 【加35元即可换购无线鼠标】商品加入购物车，查看商品上方“换购”，PC端点击“换购商品”，APP端点击“去换购”即可参与！无线鼠标E3黑红双色可换壳\n二手有售\nThinkPad京东自营旗舰店\n自营 赠\n对比 关注 加入购物车',
 '￥16999.00\n联想ThinkPad\nX1\nCarbon 2019（04CD）英特尔酷睿i7 14英寸轻薄笔记本电脑(i7-10710U 16G 1TSSD WQHD)4G版 【加35元即可换购无线鼠标】商品加入购物车，查看商品上方“换购”，PC端点击“换购商品”，APP端点击“去换购”即可参与！无线鼠标E3黑红双色可换壳\n二手有售\nThinkPad京东自营旗舰店\n自营 赠\n对比 关注 加入购物车',
 '￥14799.00\n联想ThinkPad\nX1\nCarbon 2019（02CD）英特尔酷睿i7 14英寸轻薄笔记本电脑(i7-10710U 16G 512GSSD WQHD)4G版 【加35元即可换购无线鼠标】商品加入购物车，查看商品上方“换购”，PC端点击“换购商品”，APP端点击“去换购”即可参与！无线鼠标E3黑红双色可换壳\nThinkPad京东自营旗舰店\n自营 赠\n对比 关注 加入购物车',
 '￥13999.00\n联想ThinkPad\nX1\nCarbon 2019（05CD）英特尔酷睿i7 14英寸轻薄笔记本电脑(i7-10710U 16G 512SSD FHD)4G版 【加35元即可换购无线鼠标】商品加入购物车，查看商品上方“换购”，PC端点击“换购商品”，APP端点击“去换购”即可参


-----

# 本周目标
* [猎聘PC版](https://www.liepin.com/zhaopin/)
* 上方导航有  公司行业 城市 薪资 的分页选单
* 请练习xpath抽出数据

## Xpath解析HTML

In [8]:
# A-1   单一页面
url = "https://www.liepin.com/zhaopin/?keyword=PRD"
session = HTMLSession()
r = session.get( url )

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath( \
    '//ul[@class="sojob-list"]/li')

# 预期是一个元素的列表？
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]'))
#print (主要元素[0].xpath('//div[contains(@class,"job-info")]/h3/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/a'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="text-warning"]'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span'))
#print (主要元素[3].xpath('//div[contains(@class,"job-info")]/p/time/@title'))
#print (主要元素[0].xpath('//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a'))

# 作为xpath字典，键为我要抓的牛肉名称，值为xpath
dict_xpaths={ 
    'text': {
        'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
        '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
        '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
        '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
        '职称':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司地点': '//div[contains(@class,"job-info")]/p/a',
        '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    },
    'text_content': {
    },
    'href': {
        '链结':    '//div[contains(@class,"job-info")]/h3/a', 
        '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
    }
}

def get_e_text_content(_xpath_):
    # 高级列表推导
    暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
    return(暂存结果)

def get_e_text(_xpath_):
    # 高级列表推导
    暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
    return(暂存结果)

def get_e_href(_xpath_):
    # 高级列表推导
    暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
               if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
               else "" for e in 主要元素]
    return(暂存结果)

# 只对主要元素下进行.xpath取值
数据字典 = dict()

数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

[len(v) for k,v in 数据字典.items()]

数据 = pd.DataFrame(数据字典)
数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
数据 

,edu,经验,薪水,时间,职称,公司地点,公司名称,链结,公司URL
0,本科及以上,5年以上,7-10k·12薪,2020年03月31日,成本核算岗,昆明,昆明中药厂,https://www.liepin.com/job/1927100489.shtml,https://www.liepin.com/company/8417645/
1,学历不限,5年以上,6-12k·12薪,2020年03月31日,零售经理/区域经理,南京,广东海华投资集团有限公司,https://www.liepin.com/job/1927100345.shtml,https://www.liepin.com/company/7929937/
2,大专及以上,10年以上,20-30k·12薪,2020年03月31日,厂长,,盛大金禧,https://www.liepin.com/job/1927100331.shtml,https://www.liepin.com/company/7904501/
3,本科及以上,10年以上,40-50k·12薪,2020年03月31日,项目总经理,广州-天河区,广州市番禺汇诚小额贷款股份有限公司,https://www.liepin.com/job/1927099657.shtml,https://www.liepin.com/company/10163357/
4,统招本科,3年以上,面议,2020年03月31日,法务专员（知识产权方向）,大连,枫叶教育，枫叶集团,https://www.liepin.com/job/1927099219.shtml,https://www.liepin.com/company/8295022/
5,本科及以上,8年以上,面议,2020年03月31日,风控经理,珠海-香洲区,珠海市香洲正方控股有限公司,https://www.liepin.com/job/1927099183.shtml,https://www.liepin.com/company/9669313/
6,统招本科,3年以上,15-20k·12薪,2020年03月31日,VUE/UniAPP前端高级开发,北京-小西天,联系网科技,https://www.liepin.com/job/1927099079.shtml,https://www.liepin.com/company/8441517/
7,统招本科,5年以上,面议,2020年03月31日,电气工程师,天津-津南区,国家会展中心(天津)有限责任公司,https://www.liepin.com/job/1927098585.shtml,https://www.liepin.com/company/8293934/
8,大专及以上,5年以上,7-8k·13薪,2020年03月31日,高级厂务电气技术员,天津-北辰区,莱宝真空设备(天津)有限公司,https://www.liepin.com/job/1927098265.shtml,https://www.liepin.com/company/9459219/
9,大专及以上,1年以上,15-30k·13薪,2020年03月31日,音频策划,成都,成都乐狗科技有限公司,https://www.liepin.com/job/1927098119.shtml,https://www.liepin.com/company/9728699/


In [9]:
# A-2 扩张 公司 ?  

# 先取特定元素, 精准打击其子后辈
主要元素 = r.html.xpath('//div[@data-selector="search-conditions"]')
# 预期是一个元素的列表？
print (主要元素)
print (主要元素[0])
print (主要元素[0].xpath('//dt[@class="search-title"]'))

list_search_title = 主要元素[0].xpath('//dt[@class="search-title"]')
for x in list_search_title:
    print (x.text)
    
list_search_dd = 主要元素[0].xpath('//dt[@class="search-title"]/following-sibling::dd')
for x in list_search_dd:
    print (x)  
    

公司数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                    .xpath('//div[contains(@class,"hot-comp-tags")]/a/@href')
               
公司数据选择器链结

# 但我们需要知道这些选择器链结, 对映到什麽数据
公司数据选择器链结 = r.html.xpath('//div[@data-selector="search-conditions"]')[0] \
                    .xpath('//dt[@class="search-title"]/following-sibling::dd')[0] \
                    .xpath('//div[contains(@class,"hot-comp-tags")]/a')
公司数据选择器链结

#[ x.xpath("a/@href")[0] for x in 公司数据选择器链结]
#[ x.xpath("a/text()")[0] for x in 公司数据选择器链结]
公司数据选择器链结 = { x.xpath("a/text()")[0]:x.xpath("a/@href")[0] for x in 公司数据选择器链结}
公司数据选择器链结

[<Element 'div' class=('search-conditions',) data-selector='search-conditions'>]
<Element 'div' class=('search-conditions',) data-selector='search-conditions'>
[<Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>, <Element 'dt' class=('search-title',)>]
公司：
行业：
城市：
薪资：
更多：
<Element 'dd' class=('comp-list',)>
<Element 'dd' class=('short-dd', 'select-industry') data-param='industries'>
<Element 'dd' data-param='city'>
<Element 'dd' data-param='salary'>
<Element 'dd' class=('dropdown', 'dropdown-time')>
<Element 'dd' class=('dropdown', 'dropdown-jobkind')>
<Element 'dd' class=('dropdown', 'dropdown-compscale')>
<Element 'dd' class=('dropdown', 'dropdown-compkind')>


{'中国500强': '/zhaopin/?init=-1&headckid=58d828c357a8cb19&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=155&ckid=58d828c357a8cb19&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6aa779111c1b4ca77cff3648d9dee049&d_curPage=0&d_pageSize=40&d_headId=6aa779111c1b4ca77cff3648d9dee049',
 '2018互联网300强': '/zhaopin/?init=-1&headckid=58d828c357a8cb19&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=182&ckid=58d828c357a8cb19&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6aa779111c1b4ca77cff3648d9dee049&d_curPage=0&d_pageSize=40&d_headId=6aa779111c1b4ca77cff3648d9dee049',
 '制造业500强': '/zhaopin/?init=-1&headckid=58d828c357a8cb19&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=186&ckid=58d828c357a8cb19&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6aa779111c1b4ca77cff3648d9dee049&d_curPage=0&d_pageSize=40&d_headId=6aa779111c1b4ca77cff3648d9dee049',
 'AI创新成长50强 ': '/zhaopin/?init=-1&headckid=

## 使用urllib3 解析 url 
上面的url应该触动不同的页面查询，但能不能轻松无误的拆分url并进行比较？

### urllib模块功能介绍
* urlparse 
返回的6个部分，分别是：scheme(机制)丶netloc(网络位置)丶path(路径)丶params(路径段参数)丶query(查询)丶fragment(片段)。
* parse_qs
返回query(查询)多个部分

In [10]:
# B-1 使用 urllib.parse 解析
from urllib.parse import urlparse, parse_qs
[ urlparse(x) for x in 公司数据选择器链结.values()]

[ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=58d828c357a8cb19&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=155&ckid=58d828c357a8cb19&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6aa779111c1b4ca77cff3648d9dee049&d_curPage=0&d_pageSize=40&d_headId=6aa779111c1b4ca77cff3648d9dee049', fragment=''),
 ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=58d828c357a8cb19&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=182&ckid=58d828c357a8cb19&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6aa779111c1b4ca77cff3648d9dee049&d_curPage=0&d_pageSize=40&d_headId=6aa779111c1b4ca77cff3648d9dee049', fragment=''),
 ParseResult(scheme='', netloc='', path='/zhaopin/', params='', query='init=-1&headckid=58d828c357a8cb19&flushckid=1&fromSearchBtn=2&keyword=PRD&compTag=186&ckid=58d828c357a8cb19&siTag=1B2M2Y8AsgTpgAmY7PhCfg%7EfA9rXquZc5IkJpXC-Ycixw&d_sfrom

In [11]:
# B-2 使用 pd.DataFrame进行 unuinque()相异值计量比对 
import pandas as pd
df = pd.DataFrame([ urlparse(x) for x in 公司数据选择器链结.values()])
df.info()
print(df.nunique())
df.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   scheme    6 non-null      object
 1   netloc    6 non-null      object
 2   path      6 non-null      object
 3   params    6 non-null      object
 4   query     6 non-null      object
 5   fragment  6 non-null      object
dtypes: object(6)
memory usage: 416.0+ bytes
scheme      1
netloc      1
path        1
params      1
query       6
fragment    1
dtype: int64


,scheme,netloc,path,params,query,fragment
0,,,/zhaopin/,,init=-1&headckid=58d828c357a8cb19&flushckid=1&...,


In [12]:
# B-3 针对query 再解析之 
#df_qs = pd.DataFrame([ parse_qs(x) for x in df['query'] ])
df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])
print (df_qs.nunique())
df_qs.head()
df_qs[['keyword','compTag']]

init             1
headckid         1
flushckid        1
fromSearchBtn    1
keyword          1
compTag          6
ckid             1
siTag            1
d_sfrom          1
d_ckId           1
d_curPage        1
d_pageSize       1
d_headId         1
dtype: int64


,keyword,compTag
0,PRD,155
1,PRD,182
2,PRD,186
3,PRD,189
4,PRD,130
5,PRD,156


In [13]:
# B-3-X 对 B-3 代码的字典/列表推导的分拆说明

# ----------------------------------------------
# 列表暂存 = [{k:v[0] for k,v in parse_qs(q).items()} for q in df['query'] ]
# 以下3行代碼相當於上面推導1行
列表暂存 = [] # list()
for x in df['query']: 
    列表暂存.append({k:v[0] for k,v in parse_qs(x).items()} )
    
# ----------------------------------------------
# 字典暂存 = {k:v[0] for k,v in parse_qs(x).items()}
# 以下3行代碼相當於上面推導1行
字典暂存 = dict()
for k,v in parse_qs(x).items():     # for 键,值 in 字典.items():
    字典暂存.update({k:v[0]})
    
# ----------------------------------------------
print (列表暂存, 字典暂存) 

'''python 原代碼以下不處理
df_qs = pd.DataFrame(list_query)
print (df_qs.nunique())
df_qs.head()
df_qs[['keyword','compTag']]
'''

[{'init': '-1', 'headckid': '58d828c357a8cb19', 'flushckid': '1', 'fromSearchBtn': '2', 'keyword': 'PRD', 'compTag': '155', 'ckid': '58d828c357a8cb19', 'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw', 'd_sfrom': 'search_unknown', 'd_ckId': '6aa779111c1b4ca77cff3648d9dee049', 'd_curPage': '0', 'd_pageSize': '40', 'd_headId': '6aa779111c1b4ca77cff3648d9dee049'}, {'init': '-1', 'headckid': '58d828c357a8cb19', 'flushckid': '1', 'fromSearchBtn': '2', 'keyword': 'PRD', 'compTag': '182', 'ckid': '58d828c357a8cb19', 'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw', 'd_sfrom': 'search_unknown', 'd_ckId': '6aa779111c1b4ca77cff3648d9dee049', 'd_curPage': '0', 'd_pageSize': '40', 'd_headId': '6aa779111c1b4ca77cff3648d9dee049'}, {'init': '-1', 'headckid': '58d828c357a8cb19', 'flushckid': '1', 'fromSearchBtn': '2', 'keyword': 'PRD', 'compTag': '186', 'ckid': '58d828c357a8cb19', 'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw', 'd_sfrom': 'search_unknown', 'd_ckId': '6aa77911

"python 原代碼以下不處理\ndf_qs = pd.DataFrame(list_query)\nprint (df_qs.nunique())\ndf_qs.head()\ndf_qs[['keyword','compTag']]\n"

In [14]:
# B-3-XX 整合后

列表暫存 = [] # list()
for q in df['query']: 
    字典暫存 = dict()
    for k,v in parse_qs(q).items(): # for 鍵,值 in 字典.items():
        字典暫存.update({k:v[0]})
    列表暫存.append(字典暫存)
列表暫存
# ----------------------------------------------
# 比較
# 列表暫存 = [{k:v[0] for k,v in parse_qs(q).items()} for q in df['query'] ]

[{'init': '-1',
  'headckid': '58d828c357a8cb19',
  'flushckid': '1',
  'fromSearchBtn': '2',
  'keyword': 'PRD',
  'compTag': '155',
  'ckid': '58d828c357a8cb19',
  'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw',
  'd_sfrom': 'search_unknown',
  'd_ckId': '6aa779111c1b4ca77cff3648d9dee049',
  'd_curPage': '0',
  'd_pageSize': '40',
  'd_headId': '6aa779111c1b4ca77cff3648d9dee049'},
 {'init': '-1',
  'headckid': '58d828c357a8cb19',
  'flushckid': '1',
  'fromSearchBtn': '2',
  'keyword': 'PRD',
  'compTag': '182',
  'ckid': '58d828c357a8cb19',
  'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw',
  'd_sfrom': 'search_unknown',
  'd_ckId': '6aa779111c1b4ca77cff3648d9dee049',
  'd_curPage': '0',
  'd_pageSize': '40',
  'd_headId': '6aa779111c1b4ca77cff3648d9dee049'},
 {'init': '-1',
  'headckid': '58d828c357a8cb19',
  'flushckid': '1',
  'fromSearchBtn': '2',
  'keyword': 'PRD',
  'compTag': '186',
  'ckid': '58d828c357a8cb19',
  'siTag': '1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc

### 小结
* comTag 是不同的公司选择器, 数值不样, 对映到不同类型的公司
* keyword 是搜查关键字

In [15]:
# B-4 建构 参数模板 及 字典_compTag
def parse_url_qs_for_compTag (url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

# parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])['compTag']
参数模板 = parse_url_qs_for_compTag(list(公司数据选择器链结.values())[0])
print(参数模板)
# [ parse_url_qs_for_compTag(x)['compTag'] for x in 公司数据选择器链结.values()]
[ parse_url_qs_for_compTag(x)['compTag'][0] for x in 公司数据选择器链结.values()]

字典_compTag = { k:parse_url_qs_for_compTag(v)['compTag'][0] for k,v in 公司数据选择器链结.items()}
print (字典_compTag)


{'init': ['-1'], 'headckid': ['58d828c357a8cb19'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['PRD'], 'compTag': ['155'], 'ckid': ['58d828c357a8cb19'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['6aa779111c1b4ca77cff3648d9dee049'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['6aa779111c1b4ca77cff3648d9dee049']}
{'中国500强': '155', '2018互联网300强': '182', '制造业500强': '186', 'AI创新成长50强 ': '189', '独角兽': '130', '上市公司': '156'}


In [16]:
# B-5 建构 参数模板  
def 参数模板生成(compTag , keyword ):
    参数 = 参数模板.copy()
    参数['compTag'] = compTag
    参数['keyword'] = keyword
    return (参数)

参数_compTag_用户体验 = { k:参数模板生成(compTag = [v], keyword = ['用户体验']) for k,v in 字典_compTag.items()}
print(参数_compTag_用户体验)

{'中国500强': {'init': ['-1'], 'headckid': ['58d828c357a8cb19'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['155'], 'ckid': ['58d828c357a8cb19'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['6aa779111c1b4ca77cff3648d9dee049'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['6aa779111c1b4ca77cff3648d9dee049']}, '2018互联网300强': {'init': ['-1'], 'headckid': ['58d828c357a8cb19'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['182'], 'ckid': ['58d828c357a8cb19'], 'siTag': ['1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw'], 'd_sfrom': ['search_unknown'], 'd_ckId': ['6aa779111c1b4ca77cff3648d9dee049'], 'd_curPage': ['0'], 'd_pageSize': ['40'], 'd_headId': ['6aa779111c1b4ca77cff3648d9dee049']}, '制造业500强': {'init': ['-1'], 'headckid': ['58d828c357a8cb19'], 'flushckid': ['1'], 'fromSearchBtn': ['2'], 'keyword': ['用户体验'], 'compTag': ['186'], 'ckid': ['58d828c357a8cb19'], '

## requests 生成

In [17]:
# C-1   多个页面准备测试1 中国500强
url = "https://www.liepin.com/zhaopin/"
session = HTMLSession()
payload = 参数_compTag_用户体验['中国500强']
r = session.get( url, params = payload)
r.url

'https://www.liepin.com/zhaopin/?init=-1&headckid=58d828c357a8cb19&flushckid=1&fromSearchBtn=2&keyword=%E7%94%A8%E6%88%B7%E4%BD%93%E9%AA%8C&compTag=155&ckid=58d828c357a8cb19&siTag=1B2M2Y8AsgTpgAmY7PhCfg~fA9rXquZc5IkJpXC-Ycixw&d_sfrom=search_unknown&d_ckId=6aa779111c1b4ca77cff3648d9dee049&d_curPage=0&d_pageSize=40&d_headId=6aa779111c1b4ca77cff3648d9dee049'

In [18]:
# C-2  简化 A-1   单一页面爬+解析
session = HTMLSession()

def requests_liepin( url, params):
    r = session.get( url , params = payload)

    # 先取特定元素, 精准打击其子后辈
    主要元素 = r.html.xpath( '//ul[@class="sojob-list"]/li')

    # 作为xpath字典，键为我要抓的牛肉名称，值为xpath
    dict_xpaths={ 
        'text': {
            'edu':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]',
            '经验':      '//div[contains(@class,"job-info")]/p/span[@class="edu"]/following-sibling::span',
            '薪水':    '//div[contains(@class,"job-info")]/p/span[@class="text-warning"]', 
            '时间':    '//div[contains(@class,"job-info")]/p/time/@title', 
            '职称':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司地点': '//div[contains(@class,"job-info")]/p/a',
            '公司名称': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        },
        'text_content': {
        },
        'href': {
            '链结':    '//div[contains(@class,"job-info")]/h3/a', 
            '公司URL': '//div[contains(@class,"sojob-item-main")]//p[@class="company-name"]/a', 
        }
    }

    def get_e_text_content(_xpath_):
        # 高级列表推导
        暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]
        return(暂存结果)

    def get_e_text(_xpath_):
        # 高级列表推导
        暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
        return(暂存结果)

    def get_e_href(_xpath_):
        # 高级列表推导
        暂存结果 = [list(e.xpath(_xpath_, first=True).absolute_links)[0] \
                   if len(e.xpath(_xpath_, first=True).absolute_links) >= 1  \
                   else "" for e in 主要元素]
        return(暂存结果)

    # 只对主要元素下进行.xpath取值
    数据字典 = dict()

    数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
    数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
    数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

    数据 = pd.DataFrame(数据字典)
    #数据.to_excel("20春_Web数据挖掘_week03_liepin.xlsx", sheet_name="搜查结果")
    return (数据)




In [19]:
# C-3   多个页面
url = "https://www.liepin.com/zhaopin/"

list_df = list()
for k,v in 参数_compTag_用户体验.items():
    payload = v
    df = requests_liepin( url, params = payload)
    df = df.assign (热门公司类型 = k)    
    list_df.append(df)

df_all = pd.concat(list_df)
df_all

,edu,经验,薪水,时间,职称,公司地点,公司名称,链结,公司URL,热门公司类型
0,本科及以上,3年以上,面议,2020年03月31日,大客户销售经理-北京-网易严选,北京-五道口,网易集团,https://www.liepin.com/job/1926756751.shtml,https://www.liepin.com/company/5964833/,中国500强
1,统招本科,3年以上,面议,2020年03月30日,阿里云智能事业群-数据技术专家(金融行业)-北京/杭州,杭州,阿里巴巴,https://www.liepin.com/job/1927063431.shtml,https://www.liepin.com/company/1072424/,中国500强
2,本科及以上,3年以上,面议,2020年03月27日,钉钉(Dingtalk)-搜索中心-Java开发技术专家,杭州,阿里巴巴,https://www.liepin.com/job/1926996383.shtml,https://www.liepin.com/company/1072424/,中国500强
3,大专及以上,2年以上,6-8k·13薪,2020年03月25日,员工关系专员,廊坊-广阳区,中国国际技术智力合作有限公司,https://www.liepin.com/job/1926938099.shtml,https://www.liepin.com/company/1233751/,中国500强
4,本科及以上,3年以上,25-50k·12薪,2020年03月24日,钉钉(DingTalk)-安全运营专家-安全产品及中心,杭州,阿里巴巴,https://www.liepin.com/job/1926923363.shtml,https://www.liepin.com/company/1072424/,中国500强
...,...,...,...,...,...,...,...,...,...,...
35,本科及以上,1年以上,5-8k·12薪,2020年03月26日,片区人力资源主任/专员,中山,碧桂园智慧物业服务集团股份有限公司,https://www.liepin.com/job/1919360705.shtml,https://www.liepin.com/company/8694860/,上市公司
36,本科及以上,6年以上,15-20k·13薪,2020年03月26日,法务经理/主任,深圳,中国南玻集团股份有限公司,https://www.liepin.com/job/1926955487.shtml,https://www.liepin.com/company/9091167/,上市公司
37,统招本科,10年以上,面议,2020年03月26日,CHO/HRD,上海,银科控股,https://www.liepin.com/job/1915800458.shtml,https://www.liepin.com/company/8582797/,上市公司
38,本科及以上,3年以上,20-30k·12薪,2020年03月25日,SAP 运维顾问,北京,科兴,https://www.liepin.com/job/1926949105.shtml,https://www.liepin.com/company/8593199/,上市公司


In [20]:
# C-4   输出
df_all.to_excel("20春_Web数据挖掘_week03_liepin_各热门公司类型.xlsx", sheet_name="搜查结果")

In [21]:
# C-5 Pandas  基本能力

print (df_all.nunique())
df_all[['edu']].drop_duplicates()

df_all.groupby(['公司名称','edu']).agg({"职称":"count"}).sort_values(by='职称', ascending=False)

edu         5
经验         10
薪水         74
时间         31
职称        183
公司地点       80
公司名称       59
链结        199
公司URL      59
热门公司类型      6
dtype: int64


,,职称
公司名称,edu,
华为,统招本科,30
腾讯,本科及以上,29
华为,本科及以上,16
科大讯飞,本科及以上,14
华为,硕士及以上,6
...,...,...
柳工机械,统招本科,1
华峰集团,本科及以上,1
江南布衣,大专及以上,1


In [22]:
df_all.groupby(['公司名称','edu']).agg({"职称":"count"}).sort_values(by='职称', ascending=False)

,,职称
公司名称,edu,
华为,统招本科,30
腾讯,本科及以上,29
华为,本科及以上,16
科大讯飞,本科及以上,14
华为,硕士及以上,6
...,...,...
柳工机械,统招本科,1
华峰集团,本科及以上,1
江南布衣,大专及以上,1


# 本周练习

一样反向工程解析:

## 上方界面的params参数
* 公司：v
* 行业：?
* 城市：?
* 薪资：?
## 下方界面的params参数
* 跳转到 N 页确定 ?
## 换  
* keyword
